In [3]:

# coding: utf-8

# In[1]:

from numpy import *

def get_R_o(EAngles):

    # returns rotation matrix
            
    R_o = zeros((3,3),dtype=double)
    
    euler_phi1 = EAngles[0]
    euler_Phi = EAngles[1]
    euler_phi2 = EAngles[2]
    
    R_o[0,0]= cos(euler_phi1)*cos(euler_phi2) - sin(euler_phi1)*sin(euler_phi2)*cos(euler_Phi)
    R_o[1,0]= -cos(euler_phi1)*sin(euler_phi2) - sin(euler_phi1)*cos(euler_phi2)*cos(euler_Phi)
    R_o[2,0]= sin(euler_phi1)*sin(euler_Phi)
    R_o[0,1]= sin(euler_phi1)*cos(euler_phi2) + cos(euler_phi1)*sin(euler_phi2)*cos(euler_Phi)
    R_o[1,1]= -sin(euler_phi1)*sin(euler_phi2) + cos(euler_phi1)*cos(euler_phi2)*cos(euler_Phi) 
    R_o[2,1]= -cos(euler_phi1)*sin(euler_Phi)
    R_o[0,2]= sin(euler_phi2)*sin(euler_Phi)  
    R_o[1,2]= cos(euler_phi2)*sin(euler_Phi)
    R_o[2,2]= cos(euler_Phi) 

    return asmatrix(R_o)


# In[2]:

import numpy as np
from math import sqrt

def bcrossn():
    b = np.array([[0, 1, -1],
                 [1, 0, -1],
                 [1, -1, 0],
                 [0, -1, 1],
                 [1, 0, 1],
                 [1, 1, 0],
                 [0, 1, 1],
                 [-1, 0, 1],
                 [-1, -1, 0],
                 [0, -1, -1],
                 [-1, 0, -1],
                 [-1, 1, 0]])
    
    n = np.array([[1, 1, 1],
                 [1, 1, 1],
                 [1, 1, 1],
                 [-1, 1, 1],
                 [-1, 1, 1],
                 [-1, 1, 1],
                 [1, -1, 1],
                 [1, -1, 1],
                 [1, -1, 1],
                 [1, 1, -1],
                 [1, 1, -1],
                 [1, 1, -1]])
    
    m = np.empty((12,3,3))
    q = np.empty((12,3,3))
    for i in range(12):
        Schmid = ((1 / sqrt(6)) * b[i,:].reshape((3,1)) * n[i,:])
        m[i,:,:] = .5 * (Schmid + Schmid.T)
        q[i,:,:] = .5 * (Schmid - Schmid.T)
    return m,q


# In[3]:

def vertex_stress(i):
    v = np.zeros((28,6))
    v[0] = [1, -1, 0, 0, 0, 0]
    v[1] = [0, 1, -1, 0, 0, 0]
    v[2] = [-1, 0, 1, 0, 0, 0]
    v[3] = [0, 0, 0, 1, 0, 0]
    v[4] = [0, 0, 0, 0, 1, 0]
    v[5] = [0, 0, 0, 0, 0, 1]
    v[6] = [.5, -1, .5, 0, .5, 0]
    v[7] = [.5, -1, .5, 0, -.5, 0]
    v[8] = [-1, .5, .5, .5, 0, 0]
    v[9]= [-1, .5, .5, -.5, 0, 0]
    v[10]= [.5, .5, -1, 0, 0, .5]
    v[11]= [.5, .5, -1, 0, 0, -.5]
    v[12]= [.5, 0, -.5, .5, 0, .5]
    v[13]= [.5, 0, -.5, -.5, 0, .5]
    v[14]= [.5, 0, -.5, .5, 0, -.5]
    v[15]= [.5, 0, -.5, -.5, 0, -.5]
    v[16]= [0, -.5, .5, 0, .5, .5]
    v[17]= [0, -.5, .5, 0, -.5, .5]
    v[18]= [0, -.5, .5, 0, .5, -.5]
    v[19]= [0, -.5, .5, 0, -.5, -.5]
    v[20]= [-.5, .5, 0, .5, .5, 0]
    v[21]= [-.5, .5, 0, -.5, .5, 0]
    v[22]= [-.5, .5, 0, .5, -.5, 0]
    v[23]= [-.5, .5, 0, -.5, -.5, 0]
    v[24]= [0, 0, 0, .5, .5, .5]
    v[25]= [0, 0, 0, .5, -.5, .5]
    v[26]= [0, 0, 0, -.5, .5, .5]
    v[27]= [0, 0, 0, .5, .5, .5]

    return sqrt(6) * v[i]


# In[4]:

def vect2dev_tensor(V):
    DT = np.zeros((3,3))
    DT[0,0] = (V[2] - V[1]) / 3
    DT[1,1] = (V[0] - V[2]) / 3
    DT[2,2] = (V[1] - V[0]) / 3
    DT[0,1] = V[5]
    DT[1,0] = V[5]
    DT[0,2] = V[4]
    DT[2,0] = V[4]
    DT[1,2] = V[3]
    DT[2,1] = V[3]
    return DT


# In[5]:

def BHmodel(de_c, g, i):
    dW = []
    for j in range(28):
        vs = vertex_stress(j)
        A = vs[0]
        B = vs[1]
        C = vs[2]
        F = vs[3]
        G = vs[4]
        H = vs[5]

        dW.append(-B * de_c[0,0] + A * de_c[1,1] + 2 * F * de_c[2,1] + 2 * G * de_c[2,0] + 2 * H * de_c[0,1])
    
#     print dW
    maxdW = 0
    index = -1
    for i in range(len(dW)):
        if abs(dW[i]) > maxdW:
            maxdW = abs(dW[i])
            index = i
#     print maxdW
#     print index
    if dW[index] < 0:
        S_c = -1 * vertex_stress(index)
    else:
        S_c = vertex_stress(index)

    Stensor_c = vect2dev_tensor(S_c)
    return g.T * Stensor_c * g


# In[6]:

def get_taylor(phi1,Phi,phi2):
    Bunge = np.array((phi1, Phi, phi2)).T
    
    ###################################
    de = np.array([[-.5,  0,   0],
                  [0, -.5,  0],
                  [0,  0,  1]])
    ###################################
    
    m, q = bcrossn()
#     Nc = len(Euler1)
    S = np.zeros((3,3))
    Sdotde = 0
    dedotde = 0
    eVM = 0
#     angle = np.zeros((Nc*Nc,1))
#     a = 11
    g = get_R_o(Bunge).T # I hope this works right, can't test it yet
#     print "Euler Angles (radians): ", Bunge[a], "\n"
#     print "RMatOfBunge/R_o: "
#     print g

    # print de
    de_c = g * de * g.T
    # print de_c

    Sc = BHmodel(de_c, g, 0)
    # print Sc
    Sdotde = 0
    dedotde = 0
    eVM = 0

    for k in range(3):
        for l in range(3):
            Sdotde += Sc[k,l] * de[k,l]
            dedotde += de[k,l] * de[k,l]

    eVM = sqrt((2.0 / 3.0) * dedotde)
    M = Sdotde / eVM
    return M

In [4]:
# Calculates the Taylor Factor and write it to file

natee = 0

with open('../../data/10-distance-to-avg-distance.csv', 'r') as r:
    with open('../../data/11-delta-taylor-factor.csv', 'w') as w:
        # Write the file headers
        headers = ('dadN,beta,change_in_z,crack_id,theta,x,y,z,on_crack_front,grain_id,nearest_grain_boundary_x,nearest_grain_boundary_y,'
                   'nearest_grain_boundary_z,nearest_grain_boundary_id,distance_to_grain_boundary,'
                   'vector_to_grain_boundary_x,vector_to_grain_boundary_y,vector_to_grain_boundary_z,'
                   'magnitude_of_vector_to_grain_boundary,unit_vector_to_grain_boundary_x,unit_vector_to_grain_boundary_y,'
                   'unit_vector_to_grain_boundary_z,phi1,Phi,phi2,'
                   'nearest_grain_phi1,nearest_grain_Phi,nearest_grain_phi2,'
                   'misorientation,normal_x,normal_y,normal_z,raw_angle,scaled_angle,taylor_factor,previous_dadN,'
                   'curvature_distance,distance_to_avg_distance,signed_delta_taylor_factor,delta_taylor_factor\n')
        w.write(headers)
        
        r.readline()
        
        for line in r:
            tokens = line.split(',')
        
            dadN = tokens[0]
            beta = tokens[1]
            change_in_z = tokens[2]
            crack_id = int(tokens[3])
            if natee == crack_id:
                print('crack_id: {}'.format(crack_id))
                natee += 1
            theta = float(tokens[4])

            x = float(tokens[5])
            y = float(tokens[6])
            z = float(tokens[7])
            on_crack_front = tokens[8] == 'True'
            grain_id = int(tokens[9])

            nearest_grain_boundary_x = tokens[10]
            nearest_grain_boundary_y = tokens[11]
            nearest_grain_boundary_z = tokens[12]
            nearest_grain_boundary_id = int(tokens[13])

            distance_to_grain_boundary = tokens[14]
            vector_to_grain_boundary_x = float(tokens[15])
            vector_to_grain_boundary_y = float(tokens[16])
            vector_to_grain_boundary_z = float(tokens[17])
            magnitude_of_vector_to_grain_boundary = tokens[18]
            unit_vector_to_grain_boundary_x = tokens[19]
            unit_vector_to_grain_boundary_y = tokens[20]
            unit_vector_to_grain_boundary_z = tokens[21]

            phi1 = tokens[22]
            Phi = tokens[23]
            phi2 = tokens[24]
            nearest_grain_phi1 = float(tokens[25])
            nearest_grain_Phi = float(tokens[26])
            nearest_grain_phi2 = float(tokens[27])
            misorientation = tokens[28]

            normal_x = tokens[29]
            normal_y = tokens[30]
            normal_z = tokens[31]
            raw_angle = tokens[32]
            scaled_angle = tokens[33]
            taylor_factor = float(tokens[34])
            previous_dadN = tokens[35]
            curvature_distance = tokens[36]
            distance_to_avg_distance = tokens[37].strip()
            
            neighbor_taylor_factor = get_taylor(nearest_grain_phi1, nearest_grain_Phi, nearest_grain_phi2)
            
            signed_delta_taylor_factor = taylor_factor - neighbor_taylor_factor
            delta_taylor_factor = abs(signed_delta_taylor_factor)

            # Write it to the file
            if crack_id > 0 and on_crack_front:
                to_write = '{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'\
                                    .format(dadN,beta,change_in_z,crack_id,theta,x,y,z,on_crack_front,
                                    grain_id,nearest_grain_boundary_x,nearest_grain_boundary_y,
                                    nearest_grain_boundary_z,nearest_grain_boundary_id,distance_to_grain_boundary,
                                    vector_to_grain_boundary_x,vector_to_grain_boundary_y,vector_to_grain_boundary_z,
                                    magnitude_of_vector_to_grain_boundary,unit_vector_to_grain_boundary_x,
                                    unit_vector_to_grain_boundary_y, unit_vector_to_grain_boundary_z, phi1, Phi, phi2, 
                                    nearest_grain_phi1, nearest_grain_Phi, nearest_grain_phi2, misorientation, 
                                    normal_x, normal_y, normal_z, raw_angle, scaled_angle, taylor_factor, previous_dadN, 
                                    curvature_distance, distance_to_avg_distance, signed_delta_taylor_factor, delta_taylor_factor)
                w.write(to_write)
print('done')

done
